# Predicting Company Bankruptcy using Machine Learning
By CS3244 Project Team 11  
Cao Han  |  Chin Sek Yi  |  Lim Kai Sin  |  Luo Xinming  

## Table of Contents
* [Chapter 1: Project Overview](#chapter1)
    * [1.1 Project Motivation](#section1_1)
    * [1.2 Dataset Description](#section1_2)
    * [1.3 Methodologies](#section1_3)

## Chapter 1: Project Overview <a class="anchor" id="chapter1"></a>

### 1.1 Project Motivation <a id="section1_1"></a>
In today's dynamic business landscape, exemplified by recent events such as the bankruptcy of Silicon Valley Bank, the ability to anticipate and mitigate financial risks is crucial for sustainable growth and stability. This project aims to develop a robust predictive model of company bankruptcy, leveraging advanced machine learning algorithms and financial data analysis techniques, so as to equip stakeholders with nuanced insights to confidently traverse the unpredictable landscape of financial risk.

### 1.2 Dataset Description <a id="section1_2"></a>
The dataset used in this project is about bankruptcy prediction of Polish companies. The dataset contains financial rates from one year and corresponding class label that indicates bankruptcy status 3 years after that year. The data contains 10503 instances (financial statements), with 495 representing bankrupted companies and 10008 still operating at the end of the 3-year forecasting period.

The data was collected from [Emerging Markets Information Service](http://www.securities.com), which is a database containing information on emerging markets around the world. The bankrupt companies were analyzed in the period 2000-2012, while the still operating companies were evaluated from 2007 to 2013.  

Source: [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/365/polish+companies+bankruptcy+data)

### 1.3 Methodologies <a id="section1_3"></a>
This project attempts to explore the effectiveness of a diverse array of machine learning algorithms, including logistic regression, svm, k nearest neighbors and decision trees in predicting company brankruptcy, using logistic regression (linear model) as the benchmark for comparison. The project leverages ensemble methods including bagging, boosting and random forests to amplify our model's predictive capabilities, while enriching our understanding of machine learning techniques.

#### 1.3.1 Library Loading <a id="section1_3_1"></a>


In [13]:
import requests
import zipfile
import os
from scipy.io import arff
import pandas as pd

#### 1.3.2 Dataset Collection <a id="section1_3_2"></a>

In [14]:
download_url = "https://archive.ics.uci.edu/static/public/365/polish+companies+bankruptcy+data.zip"
response = requests.get(download_url)
zip_file_path = "downloaded_file.zip"

if response.status_code == 200:
    with open(zip_file_path, "wb") as file:
        file.write(response.content)
else:
    print("Failed to download the file")

# Step 2: Data Exploration (EDA)

# Step 3: Data pre-processing 

# Step 4: Data Modelling

## 4.1 Evaluation Metrics

Our primary task is to predict bankruptcy (a classification problem). Below are the key classification metrics that we will use to evaluate our model performance:

### 4.1.1 Accuracy
Accuracy provides a ratio of correctly predicted observations to the total observations. It is especially useful when the classes are balanced with SMOTE.

**Formula**:
$$\text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}}$$

### 4.1.2 Confusion Matrix and Related Terms
The confusion matrix is a table layout that allows visualization of the performance of the algorithm, where each number in the matrix represents:
- **TP (True Positives)**: Correctly predicted positive observations.
- **TN (True Negatives)**: Correctly predicted negative observations.
- **FP (False Positives)**: Incorrectly predicted as positive.
- **FN (False Negatives)**: Incorrectly predicted as negative.

### 4.1.3 Precision, Recall, and F1-Score
These metrics offer a deeper understanding of the model's performance by taking into account data imbalances, thereby providing a more nuanced view of the accuracy across different class labels.

- **Precision**:
$$ \text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}} $$
Precision measures the accuracy of positive predictions.

- **Recall** (or Sensitivity or TPR):
$$ \text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}} $$
Recall measures the ability of a model to find all the relevant cases (all positive samples).

- **F1-Score**:
$$ \text{F1-Score} = 2 \times \left( \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} \right) $$
The F1-Score is the weighted average of Precision and Recall. Therefore, this score takes both false positives and false negatives into account. A high F1-score shows a model can classify the positive class correctly, while not misclassifying many negative classes as positive.

#### Macro Average vs Micro Average

- **Macro Average**: Macro average calculates the metric independently for each class and then takes the average (hence treating all classes equally). It is useful equal weight is given to the performance of each class, regardless of its frequency.

- **Micro Average**: Micro average aggregates the contributions of all classes to compute the average metric. In other words, Micro Average will compute the overall total TP, FP, and FN across all classes, and then use these totals to calculate the performance scores. It is useful when metrics are weighted by class size, which is ideal for imbalanced data as it reflects the contribution of each class proportionally to its size.


## 4.2 Machine Learning Models 

### Scikit-learn (sklearn)
- **KNeighborsClassifier**: This model implements the k-nearest neighbors voting algorithm.
- **LogisticRegression**: A model that applies logistic regression for binary classification tasks.
- **SVC**: Support Vector Machine classifier known for its effectiveness in high-dimensional spaces.
- **DecisionTreeClassifier**: A model that uses a decision tree for classification, useful for interpretability.
- **GradientBoostingClassifier**: An ensemble model that builds on weak prediction models to create a strong classifier.
- **RandomForestClassifier**: A meta estimator that fits a number of decision tree classifiers to various sub-samples of the dataset and uses averaging to improve predictive accuracy and control overfitting.

### Imbalanced-learn (imblearn)
Dealing with imbalanced data:

- **BalancedRandomForestClassifier**: A variation of the RandomForest that handles imbalances by adjusting weights inversely proportional to class frequencies in the input data.

### XGBoost (xgboost)

- **XGBoost**: An implementation of gradient boosted decision trees designed for speed and performance.


## 4.2 Our training approach

### 4.2.1 Simple logistic regression
We begin our modeling with a simple logistic regression to establish a baseline for benchmark performance.
This approach provides an initial overview of how well simple models can predict outcomes based on our data.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

# Step 6: Numerical results